In [68]:
from ase.io import vasp
import json 
import random
import numpy as np
import math

In [69]:
f = "Shannon_radii_dict.json"
def read_json(fjson):
    with open(fjson) as f:
        return json.load(f)
data = read_json(f)

In [70]:
# This function randomly selects some atoms (determined by user in sample_size)
def selecting_atoms(N,sample_size):
    R = [i for i in range(N)]
    Selected=[]
    for i in range(math.floor(N*sample_size)):
        ran=math.floor(random.random()*(N-i))
        Selected.append(R[ran])
        R.remove(R[ran])
    return Selected

In [71]:
# This function determines the Shannon radii according the charge state, coordination of the atoms
def Shannon_raddi(file,data,i):
    if file.symbols[i] == 'Pb':
        Radii=data['Pb']['2']['6']['only_spin']
    elif file.symbols[i] == 'N':
        Radii=data['N']['-3']['4']['only_spin']
    elif file.symbols[i] == 'C':
        Radii=data['C']['4']['6']['only_spin']
    elif file.symbols[i] == 'H':
        Radii=data['H']['1']['1']['only_spin']
    elif file.symbols[i] == 'I':
        Radii=data['I']['-1']['6']['only_spin']
    return Radii

In [73]:
#percent determines the maximum amplitude of the noise with respect to the Shannon raddi
percent = 0.05
# sample_size is the fraction of number of atoms where noise will be added
sample_size=0.2
#num_config sets the total number of disticnt configuration 
num_config=10
#input file
file = vasp.read_vasp('Ma3GuPb4I12_Opt.vasp')
#N is the total number of atoms
N=file.get_global_number_of_atoms()
# Po_i has all the position of the atoms in an array
Po_i=file.get_positions().round(5)
for k in range(num_config):
    # For every configuration, the positions are set back to the origianl ones
    Po=Po_i.copy()
    # Some atoms are randomly chosen
    Selected = selecting_atoms(N,sample_size)
    for i in Selected:
    # Noise is added to all three components of the selected atoms
        Rand = np.array([2*random.random()-1 for j in range(3)])
        Radii=Shannon_raddi(file,data,i)
        Po[i,:]+= Rand*Radii*percent
    file.set_positions(Po)
    # Save the file in the POSCAR format
    vasp.write_vasp(f"Ma3GuPb4I12_config_{k}.vasp",file,direct=True)